In [2]:
import pandas as pd
from ast import literal_eval
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from statistics import mean, median
import scipy
from sklearn.decomposition import PCA
from sklearn import preprocessing
from gprofiler import GProfiler
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
import operator
import qvalue as qv


#Reactome file containing information on pathways, the genes they contain and pathway name, also including the illumina identifier for the genes.

reactome = pd.read_csv('../data/reactome.csv', sep=',', index_col = 0)

def read_reactome(file_name, gene_name_start = "ENSG0"):
    df = pd.read_csv(file_name, sep='\t', header=None)
    
    if gene_name_start == None:
        sub_df = df
    else:
        subset_vec = df[0].str.startswith(gene_name_start)
        sub_df = df.loc[subset_vec]
    
    genes_df = sub_df.groupby(1)[0].apply(list)
    names_df = sub_df.groupby(1)[3].max()
    
    out_df = pd.concat([genes_df,names_df], axis=1)
    out_df.columns = ['genes', 'pathway_name']
    
    return out_df

low_level = read_reactome('../data/Ensembl2Reactome_All_Levels.txt.gz')

def my_pca(df, n_pc=1, normalize=True):
    df = df.dropna(axis = 0, how = 'all')#redundant, but keeping it just in case
    X = df.values.T
    if normalize:
        X2 = preprocessing.scale(X)
    else:
        X2 = X
    pca = PCA(n_components = n_pc)
    pca.fit(X2)
    my_pca.pca = pca  #needed for components
    Xnew = pca.fit_transform(X2)
    out_df = pd.DataFrame(Xnew.transpose(), index=list(range(1,n_pc+1)), columns=df.columns)
    out_df = out_df.transpose()
    
    return out_df, my_pca.pca.components_, my_pca.pca.explained_variance_ratio_

#Importing metabric dataset, dividing up what is clinical/expression data and changing the type of the expression columns to float
metabric_data = pd.read_csv('../data/metabric.csv.gz')
#clinical_data = metabric_data.iloc[:27, :]
expression_data = metabric_data.iloc[27:,:]

#print(expression_data.columns)
dtypedict = {}
for i in expression_data.columns[1:]:
    dtypedict[i] = 'float32'
expression_data = expression_data.astype(dtypedict)



new_clinical_patient = pd.read_csv('../data/data_clinical_patient.txt', sep='\t', index_col=0).iloc[4:]
new_clinical_sample = pd.read_csv('../data/data_clinical_sample.txt', sep='\t', index_col=0).iloc[4:]
new_clinical = pd.concat([new_clinical_patient, new_clinical_sample.reindex(new_clinical_patient.index)], axis=1)
new_clinical['Triple Neg'] = new_clinical.apply(lambda row: True if ((row['ER Status'] == 'Negative') 
                                                                     and (row['PR Status'] == 'Negative') 
                                                                     and (row['HER2 Status'] == 'Negative')) else False, axis = 1)

new_clinical['ER-/PR-/HER2+'] = new_clinical.apply(lambda row: True if ((row['ER Status'] == 'Negative') 
                                                                     and (row['PR Status'] == 'Negative') 
                                                                     and (row['HER2 Status'] == 'Positive')) else False, axis = 1)





genes = expression_data['Unnamed: 0'].values.tolist()

gp = GProfiler(return_dataframe = True)
gp = gp.convert(organism='hsapiens',
          query=genes)

gp = gp.loc[gp['n_converted'] == 1]
gp = gp.loc[gp['name'] != 'None']
gp = gp.set_index('incoming')
gprofiler_names = gp
gprofiler_names

dataset = expression_data.set_index('Unnamed: 0') #gene_patient
pca_per_pathway = pd.DataFrame(index=expression_data.columns)

real_gene_names = pd.read_csv('../data/illumina2symbol.txt', sep="\t", index_col = 0)


In [3]:
genes_components_per_pathway = {} #nested dictionary where the 'outer dictionary' is the pathway names as keys and values are 
                                  #another dictionary with genes as keys and components as values

for pathway in reactome.index:
    genes = reactome.loc[pathway, "illumina"]
    genes = literal_eval(genes)
    genes = list(filter(lambda a: a != 'NaN', genes))
    pathwaydata = dataset.loc[genes]
    if pathwaydata.index.empty == True:
        pass
    else:
        pathwaydata = pathwaydata.dropna(axis = 0, how = 'any') #has to be done so the lists match, this makes the dropna in my_pca function obsolete
        presentgenes = pathwaydata.index.values.tolist()
        if len(presentgenes) <= 1:
            pass
        else:
            res, components, explained_variance = my_pca(pathwaydata)
            print(res)
            pathwayname = reactome.loc[pathway, 'pathway_name']
            pca_per_pathway[pathwayname] = res

            components = components.tolist()[0]
            innerdict = {}
            for i in range(0, len(presentgenes)):
                component = components[i]
                gene = genes[i]
                if gene in real_gene_names.index:
                    real_name = real_gene_names.loc[gene, "symbol"]
                    innerdict[real_name] = component
                elif gene in gprofiler_names.index:
                    real_name = gprofiler_names.loc[gene, 'name']
                    innerdict[real_name] = component
                else:
                    innerdict[gene] = component
            sorted_innerdict = sorted(innerdict.items(), key = operator.itemgetter(1), reverse = True)
            genes_components_per_pathway[pathwayname] = [sorted_innerdict, explained_variance.flat[0]]

pca_per_pathway = pca_per_pathway.iloc[1:]

                1
MB-0002 -0.464917
MB-0008  1.135679
MB-0010 -0.016407
MB-0035 -0.045985
MB-0036 -1.107911
MB-0050  1.003818
MB-0059 -0.473151
MB-0060  0.681487
MB-0066  0.539735
MB-0101  1.391564
MB-0102  0.975991
MB-0106 -1.716133
MB-0112  0.721624
MB-0115  2.891249
MB-0117 -1.218243
MB-0119  0.324922
MB-0120 -1.282450
MB-0121  0.066523
MB-0122  0.241644
MB-0123  0.632205
MB-0126  0.430238
MB-0131  0.059169
MB-0133 -0.570783
MB-0135 -0.117803
MB-0136  0.874497
MB-0138  1.158647
MB-0139  2.100204
MB-0140 -0.166483
MB-0142 -0.881734
MB-0143  0.237473
...           ...
MB-7264  0.052617
MB-7266 -2.427949
MB-7267  1.027190
MB-7268  0.899953
MB-7269 -1.542248
MB-7270 -0.818144
MB-7273  0.133797
MB-7275  0.119507
MB-7276  1.452775
MB-7277 -0.836270
MB-7278 -0.750753
MB-7279 -0.591865
MB-7280  3.168238
MB-7281 -0.708822
MB-7283 -1.524783
MB-7284 -1.585520
MB-7285  2.112240
MB-7286  0.464737
MB-7287 -0.461482
MB-7288  0.084160
MB-7289  0.948162
MB-7291  1.676028
MB-7292 -0.991138
MB-7293  0

                1
MB-0002  4.655595
MB-0008  8.272937
MB-0010  5.898238
MB-0035  0.494913
MB-0036  4.147962
MB-0050 -1.933750
MB-0059  0.905942
MB-0060 -1.130984
MB-0066 -1.228741
MB-0101  6.086675
MB-0102  3.120503
MB-0106  2.340538
MB-0112  6.462940
MB-0115  3.692867
MB-0117 -0.745161
MB-0119  7.126931
MB-0120  8.332044
MB-0121  1.312075
MB-0122  1.857699
MB-0123  6.007674
MB-0126  6.382888
MB-0131  1.849582
MB-0133 -3.666937
MB-0135  0.420730
MB-0136 -4.222499
MB-0138  1.301518
MB-0139 -2.491193
MB-0140  2.869867
MB-0142  4.153456
MB-0143  1.076458
...           ...
MB-7264  3.076340
MB-7266  7.640171
MB-7267  4.764864
MB-7268  0.048722
MB-7269  2.775679
MB-7270  9.691809
MB-7273 -0.342195
MB-7275  0.164049
MB-7276 -2.698676
MB-7277 -1.550795
MB-7278  0.962199
MB-7279 -2.671319
MB-7280  3.281408
MB-7281 -1.002801
MB-7283 -2.440549
MB-7284 -3.839313
MB-7285  4.370537
MB-7286 -2.683915
MB-7287  2.022614
MB-7288  3.261115
MB-7289 -0.795144
MB-7291 -1.753836
MB-7292  6.284089
MB-7293 -1

                1
MB-0002 -0.914643
MB-0008  0.457899
MB-0010 -2.114125
MB-0035  0.535428
MB-0036 -0.270441
MB-0050  1.957522
MB-0059 -0.177012
MB-0060  0.233749
MB-0066 -1.427827
MB-0101 -0.919566
MB-0102 -0.418380
MB-0106  0.001737
MB-0112 -0.955717
MB-0115  2.179709
MB-0117  0.752168
MB-0119  0.309470
MB-0120 -1.337442
MB-0121  0.576469
MB-0122 -0.080777
MB-0123 -0.468607
MB-0126 -0.151944
MB-0131  0.592908
MB-0133  0.309528
MB-0135 -0.146900
MB-0136 -1.174273
MB-0138 -0.209948
MB-0139  0.100591
MB-0140  0.920868
MB-0142 -2.338977
MB-0143 -0.386166
...           ...
MB-7264 -1.394789
MB-7266 -0.134910
MB-7267  1.489459
MB-7268  0.485104
MB-7269 -0.560243
MB-7270  3.984455
MB-7273  1.437956
MB-7275  0.154371
MB-7276  0.761561
MB-7277  0.264398
MB-7278 -0.775760
MB-7279 -0.367922
MB-7280 -1.047813
MB-7281 -0.417035
MB-7283 -0.578973
MB-7284  0.224464
MB-7285  0.265897
MB-7286  0.849217
MB-7287  2.224647
MB-7288  0.046342
MB-7289  0.093805
MB-7291  0.578136
MB-7292 -1.204709
MB-7293 -1

                1
MB-0002  0.911213
MB-0008 -0.274624
MB-0010 -0.486575
MB-0035  1.666568
MB-0036  0.454627
MB-0050 -2.631669
MB-0059  0.498259
MB-0060  0.863548
MB-0066 -0.298729
MB-0101 -0.948458
MB-0102 -0.972993
MB-0106  0.088826
MB-0112 -2.478409
MB-0115 -2.419760
MB-0117 -0.014287
MB-0119  0.321138
MB-0120 -0.354611
MB-0121 -2.073063
MB-0122 -0.940431
MB-0123 -3.376586
MB-0126 -1.089355
MB-0131 -1.178041
MB-0133 -0.988896
MB-0135  0.039874
MB-0136 -2.436286
MB-0138 -0.284498
MB-0139 -2.429222
MB-0140 -1.366166
MB-0142  0.701744
MB-0143  0.308342
...           ...
MB-7264 -0.161194
MB-7266  2.927535
MB-7267  0.048698
MB-7268  0.227454
MB-7269  0.073972
MB-7270  0.871258
MB-7273 -0.918142
MB-7275 -3.327797
MB-7276  1.933581
MB-7277 -1.132358
MB-7278  0.612683
MB-7279  3.943223
MB-7280  0.053620
MB-7281  0.453760
MB-7283  3.615059
MB-7284 -0.490310
MB-7285 -3.365434
MB-7286 -0.991126
MB-7287 -0.828689
MB-7288 -2.184088
MB-7289 -3.948163
MB-7291 -5.250158
MB-7292 -1.007844
MB-7293 -3

                1
MB-0002 -0.892668
MB-0008 -1.226213
MB-0010 -1.158713
MB-0035  1.162846
MB-0036 -1.259422
MB-0050  0.167910
MB-0059  0.270095
MB-0060  2.323076
MB-0066 -0.654430
MB-0101 -0.149310
MB-0102 -0.195287
MB-0106 -0.620239
MB-0112  2.065805
MB-0115 -0.247985
MB-0117 -1.773532
MB-0119  0.523752
MB-0120 -0.001244
MB-0121  0.330842
MB-0122 -0.826070
MB-0123 -0.130823
MB-0126  0.678243
MB-0131  0.246927
MB-0133 -0.003485
MB-0135  2.757035
MB-0136 -0.583655
MB-0138  0.451847
MB-0139  0.819616
MB-0140 -0.431999
MB-0142 -0.103400
MB-0143 -0.459040
...           ...
MB-7264 -0.321082
MB-7266  1.978980
MB-7267 -0.256157
MB-7268 -1.639607
MB-7269  0.124551
MB-7270 -0.788942
MB-7273 -0.737821
MB-7275 -1.324277
MB-7276 -0.175625
MB-7277 -0.545784
MB-7278 -1.240514
MB-7279 -0.372693
MB-7280 -0.141970
MB-7281 -0.280868
MB-7283 -2.113511
MB-7284  0.308062
MB-7285 -1.548719
MB-7286  0.584588
MB-7287 -0.380072
MB-7288 -0.677379
MB-7289 -0.604604
MB-7291  0.685329
MB-7292 -1.243540
MB-7293 -0

...

                 1
MB-0002  -2.662757
MB-0008   1.391281
MB-0010   5.411535
MB-0035  -2.144725
MB-0036  -3.695115
MB-0050  -6.096961
MB-0059  -1.755801
MB-0060  11.623671
MB-0066  -1.262459
MB-0101  -5.285557
MB-0102   3.262805
MB-0106  -7.047092
MB-0112  -5.908285
MB-0115  10.449247
MB-0117  -2.930991
MB-0119  -0.534535
MB-0120   1.233338
MB-0121  -3.770648
MB-0122  -6.282298
MB-0123   7.291825
MB-0126  -4.822095
MB-0131  -2.040207
MB-0133  -7.760887
MB-0135   2.824555
MB-0136  -2.700914
MB-0138  -4.561206
MB-0139  -8.982806
MB-0140  -0.761359
MB-0142  -8.081692
MB-0143  -3.874899
...            ...
MB-7264  -1.165518
MB-7266  -3.497790
MB-7267  13.469613
MB-7268   0.092873
MB-7269   8.318882
MB-7270   9.718987
MB-7273   2.705807
MB-7275  -1.074541
MB-7276  -3.533899
MB-7277  -4.598416
MB-7278  -6.437747
MB-7279   4.528735
MB-7280  -2.196056
MB-7281   1.399524
MB-7283   3.046276
MB-7284  -4.359779
MB-7285  -2.701357
MB-7286  -0.481298
MB-7287   3.108340
MB-7288  -0.776779
MB-7289  -6.

                1
MB-0002 -0.066754
MB-0008 -0.707656
MB-0010  0.981737
MB-0035  0.563121
MB-0036  0.056669
MB-0050 -0.368492
MB-0059 -0.372931
MB-0060 -0.394453
MB-0066  0.502121
MB-0101 -1.026723
MB-0102 -0.734614
MB-0106  0.472311
MB-0112 -1.370066
MB-0115 -0.905843
MB-0117 -0.121076
MB-0119 -0.416085
MB-0120 -0.366520
MB-0121  0.189823
MB-0122  0.160486
MB-0123  0.574068
MB-0126 -0.793296
MB-0131  0.148436
MB-0133 -4.457085
MB-0135 -0.802381
MB-0136 -0.936826
MB-0138 -1.160225
MB-0139 -1.472892
MB-0140  0.347902
MB-0142  0.552486
MB-0143  0.205633
...           ...
MB-7264 -0.675158
MB-7266  1.964212
MB-7267 -0.291931
MB-7268  0.738586
MB-7269  2.390437
MB-7270  3.287606
MB-7273 -0.159833
MB-7275 -0.368932
MB-7276 -0.183986
MB-7277  0.650010
MB-7278  0.947672
MB-7279 -1.548730
MB-7280  0.587722
MB-7281  0.323010
MB-7283  0.180580
MB-7284 -0.428157
MB-7285  0.762981
MB-7286 -0.838387
MB-7287 -1.477246
MB-7288  0.588074
MB-7289 -2.679451
MB-7291  0.187251
MB-7292 -0.875249
MB-7293 -0

                1
MB-0002 -0.046436
MB-0008 -0.981780
MB-0010 -1.392278
MB-0035  0.356683
MB-0036 -0.019049
MB-0050 -0.393384
MB-0059 -0.740146
MB-0060 -0.523416
MB-0066 -1.615120
MB-0101 -1.177059
MB-0102 -1.533183
MB-0106 -1.367105
MB-0112 -0.642261
MB-0115  1.946851
MB-0117 -1.574957
MB-0119  0.761089
MB-0120 -0.147227
MB-0121 -1.243026
MB-0122 -2.350644
MB-0123 -2.372111
MB-0126 -2.122288
MB-0131  0.474389
MB-0133  0.466539
MB-0135 -3.151213
MB-0136 -1.794344
MB-0138 -1.336001
MB-0139 -0.330517
MB-0140 -2.441910
MB-0142 -0.398403
MB-0143  1.185841
...           ...
MB-7264 -1.713190
MB-7266 -2.294984
MB-7267  3.023445
MB-7268 -0.669553
MB-7269  1.516402
MB-7270  0.878197
MB-7273  2.041140
MB-7275 -2.324992
MB-7276 -3.045177
MB-7277  0.201348
MB-7278 -3.514461
MB-7279  2.912768
MB-7280  0.751744
MB-7281  0.206936
MB-7283  1.416558
MB-7284  0.299713
MB-7285 -1.774648
MB-7286 -2.092371
MB-7287  1.339715
MB-7288 -2.582124
MB-7289 -1.527943
MB-7291 -0.582931
MB-7292 -2.707255
MB-7293 -1

                 1
MB-0002  -0.181211
MB-0008  -6.103691
MB-0010  -4.429471
MB-0035   2.307187
MB-0036   0.652173
MB-0050  -1.234765
MB-0059  -4.292349
MB-0060  -7.273610
MB-0066  -7.044351
MB-0101  -1.349736
MB-0102  -3.961125
MB-0106   2.509513
MB-0112  -3.463130
MB-0115  -5.246093
MB-0117   0.818814
MB-0119  -2.911411
MB-0120  -1.535758
MB-0121  -6.407050
MB-0122  -4.043550
MB-0123  -9.242506
MB-0126  -0.852170
MB-0131  -1.802845
MB-0133   2.981785
MB-0135  -3.796857
MB-0136  -5.329271
MB-0138   2.631018
MB-0139  -0.110126
MB-0140  -1.463625
MB-0142   1.139714
MB-0143  -1.103453
...            ...
MB-7264   2.978943
MB-7266   0.236273
MB-7267   1.234850
MB-7268  -2.645638
MB-7269  -2.246198
MB-7270   0.104618
MB-7273  -6.583871
MB-7275  -6.027481
MB-7276  -0.924477
MB-7277  -2.903475
MB-7278   2.780069
MB-7279   7.453162
MB-7280   0.306340
MB-7281   0.093179
MB-7283   5.662927
MB-7284  -0.075522
MB-7285  -5.234865
MB-7286  -2.372421
MB-7287  -7.005775
MB-7288  -9.528717
MB-7289  -5.

                1
MB-0002 -1.718509
MB-0008  0.911397
MB-0010 -1.831648
MB-0035  2.369228
MB-0036 -1.141987
MB-0050  1.390615
MB-0059  1.175209
MB-0060  0.211785
MB-0066 -0.615689
MB-0101 -2.617655
MB-0102 -3.004801
MB-0106  1.725541
MB-0112 -1.900301
MB-0115  1.017730
MB-0117  1.396368
MB-0119 -2.474463
MB-0120 -1.412317
MB-0121 -2.393805
MB-0122 -0.132885
MB-0123 -3.345710
MB-0126  0.015671
MB-0131 -0.520465
MB-0133  2.628677
MB-0135 -2.965663
MB-0136 -2.120234
MB-0138  1.111477
MB-0139  1.756205
MB-0140 -4.762920
MB-0142 -0.582871
MB-0143 -0.254639
...           ...
MB-7264 -1.209314
MB-7266  0.316529
MB-7267 -3.890248
MB-7268  0.841505
MB-7269  0.578352
MB-7270 -2.605378
MB-7273 -2.158523
MB-7275 -2.593177
MB-7276 -1.464335
MB-7277 -1.670092
MB-7278 -1.164353
MB-7279  2.362302
MB-7280 -2.486317
MB-7281  0.122072
MB-7283  2.586263
MB-7284  3.582652
MB-7285 -3.616637
MB-7286  1.824788
MB-7287 -2.428679
MB-7288 -3.187387
MB-7289 -2.264110
MB-7291 -3.730553
MB-7292 -4.864984
MB-7293 -4

In [4]:
full_df = pd.concat([pca_per_pathway, new_clinical.reindex(pca_per_pathway.index)], axis= 1)

In [ ]:
from scipy.stats import ttest_ind, mannwhitneyu
import qvalue as qv

clusterframes = {}

grouped_by_cluster = full_df.groupby('Integrative Cluster')

for group in grouped_by_cluster:
    df_cluster = pd.DataFrame(index=full_df.iloc[:,:-33].columns)
    groupname = group[0]
    df = group[1].iloc[:,:-33]
    group2_df = full_df[full_df['Integrative Cluster'] != groupname].iloc[:,:-33]
    pvaluelist = []
    group1_mean_list = []
    group2_mean_list = []
    for pathway in df:
        group = df[pathway]
        group2 = group2_df[pathway]
        test = mannwhitneyu(group, group2)
        pvaluelist.append(test[1])
        group_mean = group.mean()
        group1_mean_list.append(group_mean)
        group2_mean = group2.mean()
        group2_mean_list.append(group2_mean) 
        
    
    df_cluster[f'Cluster {groupname}'] = group1_mean_list
    df_cluster['Other clusters'] = group2_mean_list
    df_cluster['Fold Change'] = np.log2(abs(df_cluster[f'Cluster {groupname}'])) - np.log2(abs(df_cluster['Other clusters']))
    
    
    df_cluster['p-values'] = pvaluelist
    qv.qvalues(df_cluster, 'p-values', f'cluster {groupname} qvalues')
    df_cluster['p-values'] = -np.log10(df_cluster['p-values'])
    df_cluster[f'cluster {groupname} qvalues'] = -np.log10(df_cluster[f'cluster {groupname} qvalues'])
    print(groupname)
    clusterframes[groupname] = df_cluster
        
        
clusterframes

In [5]:
gene_expression_per_pathway = pd.DataFrame(index=expression_data.columns)
for pathway in reactome.index:
    genes = reactome.loc[pathway, "illumina"]
    genes = literal_eval(genes)
    genes = list(filter(lambda a: a != 'NaN', genes))
    pathwaydata = dataset.loc[genes]
    pathwaydata = pathwaydata.dropna(axis = 0, how = 'any') #has to be done so the lists match, this makes the dropna in my_pca function obsolete
    presentgenes = pathwaydata.index.values.tolist()
    if len(presentgenes) <= 1:
        pass
    else:
        X = pathwaydata.values.T
        X2 = preprocessing.scale(X)
        pathwayname = reactome.loc[pathway, 'pathway_name']
        out_df = pd.DataFrame(X2, columns = presentgenes, index = dataset.columns)
        gene_expression_series = out_df.mean(axis=1)
        gene_expression_per_pathway[pathwayname] = gene_expression_series
        
gene_expression_per_pathway = gene_expression_per_pathway[1:]

In [6]:
full_GE_df = pd.concat([gene_expression_per_pathway, new_clinical.reindex(gene_expression_per_pathway.index)], axis=1)

In [7]:
from scipy.stats import ttest_ind, mannwhitneyu
import qvalue as qv

clusterframes_GE = {}

grouped_by_cluster = full_GE_df.groupby('Integrative Cluster')

for group in grouped_by_cluster:
    df_cluster = pd.DataFrame(index=full_GE_df.iloc[:,:-33].columns)
    groupname = group[0]
    df = group[1].iloc[:,:-33]
    group2_df = full_GE_df[full_GE_df['Integrative Cluster'] != groupname].iloc[:,:-33]
    pvaluelist = []
    group1_mean_list = []
    group2_mean_list = []
    for pathway in df:
        group = df[pathway]
        group2 = group2_df[pathway]
        test = mannwhitneyu(group, group2)
        pvaluelist.append(test[1])
        group_mean = group.mean()
        group1_mean_list.append(group_mean)
        group2_mean = group2.mean()
        group2_mean_list.append(group2_mean) 
        
    
    df_cluster[f'Cluster {groupname}'] = group1_mean_list
    df_cluster['Other clusters'] = group2_mean_list
    df_cluster['Fold Change'] = np.log2(abs(df_cluster[f'Cluster {groupname}'])) - np.log2(abs(df_cluster['Other clusters']))
    
    
    df_cluster['p-values'] = pvaluelist
    qv.qvalues(df_cluster, 'p-values', f'cluster {groupname} qvalues')
    df_cluster['p-values'] = -np.log10(df_cluster['p-values'])
    df_cluster[f'cluster {groupname} qvalues'] = -np.log10(df_cluster[f'cluster {groupname} qvalues'])
    print(groupname)
    clusterframes_GE[groupname] = df_cluster
        
        
clusterframes_GE

1
10
2
3
4ER+
4ER-
5
6
7
8
9


{'1':                                                     Cluster 1  Other clusters  \
 Base Excision Repair                                 0.341325       -0.025604   
 Estrogen-dependent gene expression                   0.269920       -0.020248   
 Chromosome Maintenance                               0.366366       -0.027482   
 Telomere Maintenance                                 0.380798       -0.028565   
 RMTs methylate histone arginines                     0.370195       -0.027770   
 Nucleosome assembly                                  0.419918       -0.031500   
 Deposition of new CENPA-containing nucleosomes ...   0.419918       -0.031500   
 HCMV Late Events                                     0.343341       -0.025755   
 Meiotic recombination                                0.371719       -0.027884   
 Transcriptional regulation by small RNAs             0.349776       -0.026238   
 Meiosis                                              0.297699       -0.022331   
 ESR-mediat

In [8]:
for i in clusterframes:
    print(clusterframes[i][f'cluster {i} qvalues'])
    print('')

NameError: name 'clusterframes' is not defined

In [ ]:
for i in clusterframes_GE:
    print(clusterframes_GE[i][f'cluster {i} qvalues'])
    print('')

In [ ]:
gsea_test_df = pd.concat([clusterframes['1']['cluster 1 qvalues'], clusterframes_GE['1']['cluster 1 qvalues']], axis=1)
gsea_test_df.columns = ['PCA', 'GE'] 
gsea_test_df['C'] = np.arange(len(gsea_test_df))
gsea_test_df

In [9]:
names = gp.get_library_name(database='Human')
print(names)

AttributeError: 'DataFrame' object has no attribute 'get_library_name'

In [13]:
expression_data.index = expression_data['Unnamed: 0']
expression_data = expression_data.iloc[:,1:]
expression_data = expression_data.rename(index= lambda x: gprofiler_names.loc[x, 'name'] if x in gprofiler_names.index else x)
expression_data = expression_data.rename(index= lambda x: real_gene_names.loc[x, 'symbol'] if x in real_gene_names.index else x)

expression_data

,MB-0002,MB-0008,MB-0010,MB-0035,MB-0036,MB-0050,MB-0059,MB-0060,MB-0066,MB-0101,...,MB-7289,MB-7291,MB-7292,MB-7293,MB-7294,MB-7295,MB-7296,MB-7297,MB-7298,MB-7299
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
RERE,9.013876,8.050127,8.385715,9.613113,9.394619,9.441010,8.726861,8.326064,8.275620,8.631910,...,8.971176,8.794541,8.154239,8.607796,9.346988,8.589375,8.402660,8.236918,8.376571,9.283964
ILMN_1893287,5.416854,5.434174,5.395648,5.206674,5.468673,5.341939,5.402681,5.544091,5.602178,5.363058,...,5.361251,5.424791,5.431274,5.323137,5.446764,5.419323,5.364424,5.350378,5.426071,5.332271
ILMN_1736104,5.303627,5.436292,5.670387,5.364256,5.285834,5.244118,5.357460,5.275894,5.279491,5.377604,...,5.545247,5.529074,5.229988,5.598631,5.386682,5.140903,5.147419,5.353369,5.357909,5.435157
RNF165,5.748717,5.530582,5.453305,5.339252,7.558167,7.150190,5.253296,7.498247,7.162908,5.873051,...,6.551446,6.358050,5.640045,6.053629,6.126151,6.002483,6.104059,5.402870,5.617954,7.137196
ILMN_1854015,5.666221,5.864693,5.782562,5.917886,5.641804,5.614012,5.893135,5.718247,5.839617,5.680772,...,5.779039,5.703303,5.891194,5.783599,5.829811,5.678971,5.629540,5.909581,5.541678,5.891938
ILMN_1904757,5.699735,5.303667,5.509027,5.264163,5.463810,5.499392,5.269590,5.389881,5.277771,5.355287,...,5.428968,5.363959,5.496578,5.546106,5.267130,5.116077,5.443954,5.533742,5.375647,5.221194
DLEC1,5.470781,5.745757,5.120902,5.327254,5.360535,5.493600,5.478526,5.469750,5.438557,5.538726,...,5.404373,5.322591,5.628501,5.385113,5.526006,5.356171,5.632466,5.641439,5.508103,5.359483
ARAFP2,5.419119,5.149618,5.225599,5.458613,5.217312,5.340619,5.367339,5.397219,5.374707,5.392478,...,5.178240,5.371097,5.498485,5.414573,5.422006,4.996112,5.224513,5.468793,5.305559,5.549350
PHF7,5.611212,5.934570,6.061952,6.443704,6.627111,5.489711,5.739219,5.477843,5.764232,5.445784,...,5.588633,5.640327,5.750093,5.616321,6.203116,5.802742,5.256086,5.571898,5.631592,5.462977


In [ ]:
import gseapy as gp
gseapatients = expression_data.columns.tolist()
new_clinical_patients = new_clinical.loc[gseapatients]
classes = ['cluster 1' if patient == '1' else 'not' for patient in new_clinical_patients['Integrative Cluster']]
#print(classes)
#expression_data.dropna(how='any', axis=1)
print(len(classes))
print(expression_data.shape)

gs_res = gp.gsea(data = expression_data, 
                 gene_sets ='Reactome_2016',
                 cls = classes)

1992
(48803, 1992)


2020-03-09 17:10:20,979 Warning: dropping duplicated gene names, only keep the first values
2020-03-09 17:10:23,889 Warning: Input data contains NA, filled NA with 0


In [ ]:
%tb

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
for i in clusterframes:
    
    PCA_array = clusterframes[i][f'cluster {i} qvalues']
    GE_array = clusterframes_GE[i][f'cluster {i} qvalues']
    df = pd.concat([PCA_array, GE_array], axis=1)
    df.columns = ['PCA', 'GE']
    similarity = cosine_similarity(df.PCA.values.reshape(1, -1), df.GE.values.reshape(1, -1))
    print(similarity[0][0])